# Delete

We are going to see how to delete one or multiple rows of a table based on some filtering criteria

## Imports

In [44]:
import psycopg2
import numpy as np
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
from pathlib import Path
import tifffile as tiff

## Connection

In [45]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## Loading the table from where we want to delete to check

In [46]:
table_name = 'samples'

data = qrs.get_data(table_name)

data

,id_sample,name_sample,panel_id_sample
0,1,JI_4,1


## Filtering to decide what to delete

We want to delete all the measurements with the metadata key signal_type_measurement as amplitude.

In [41]:
filtered_data = data[data['id_sample']>1]

filtered_data

,id_sample,name_sample,panel_id_sample
1,2,JI_5,1
2,3,JI_7,1
3,4,JI_8,1
4,5,JI_9,1
5,6,JI_10,1
6,7,JI_11,1
7,8,JI_12,1
8,10,prueba,3


## Deleting query

Now that we have only the rows that meet our deleting conditions, we can delete the measurements and their metadata with their ids

In [42]:
indexes = filtered_data['id_sample'].values.tolist() #get the indexes to delete

# Convert indexes to a tuple for SQL query
if len(indexes) == 0:
    print("No indexes to delete.")
elif len(indexes) == 1:
    indexes = tuple((indexes[0],indexes[0]))  # Ensure tuple format for single index
else:
    indexes = tuple(indexes)


sql = f'delete from {table_name} where {table_name}.id in {indexes}'
print(sql)

# Add RESTART IDENTITY command
reset_sql = f'ALTER TABLE {table_name} RESTART IDENTITY'
print(f"Reset identity SQL: {reset_sql}")

delete from samples where samples.id in (2, 3, 4, 5, 6, 7, 8, 10)
Reset identity SQL: ALTER TABLE samples RESTART IDENTITY


In [43]:
# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Execute the SQL statement
cursor.execute(sql)

cursor.execute('COMMIT')

cursor.close()

## Check

If you execute the whole code again and the filtered table is empty, the delete whas succesfull.